Half-space Trees (online)
- Half-Space Trees (window size, ensemble size)
- - **In progress**

In [1]:
from river import anomaly

In [ ]:


hst = anomaly.HalfSpaceTrees(n_trees=5, height=3, window_size=3, seed=10)

for x in X[:3]:
    hst = hst.learn_one({'x': x})

for x in X:
    features = {'x': x}
    hst = hst.learn_one(features)
    print(f'Anomaly score for x={x:.3f}: {hst.score_one(features):.3f}')

In [36]:
from river import compose
from river import datasets
from river import metrics
from river import preprocessing

import sys
sys.path.append('../../')
from feature.selection import load_td_feat, load_fd_feat, METADATA_COLUMNS_ALL
from feature.models import fault_labeling, filter_out_metadata_columns


model = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    anomaly.HalfSpaceTrees(seed=10)
)

auc = metrics.ROCAUC()

FEATURES_PATH =  '../../datasets/features_data/'
FAULT_CLASSES = {
    'normal': 'N',
    'imbalance': 'I',
    'horizontal-misalignment': 'HM',
    'vertical-misalignment': 'VM'
}

stream = load_fd_feat(['az'], path=FEATURES_PATH)
stream = fault_labeling(stream, FAULT_CLASSES)

for idx, row in stream.iterrows():
    x = {k: v for k, v in dict(row).items() if k not in METADATA_COLUMNS_ALL}
    y = row['anomaly']
    score = model.score_one(x)
    model = model.learn_one(x)
    auc = auc.update(y, score)

auc   # if is less than 50% then invert predictions False <-> True

ROCAUC: 39.24%